In [2]:
import open3d as o3d
import os
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm
import math
import json
import random
import uuid
import ifcopenshell
from ifcopenshell import template
import ifcopenshell.geom

from src.visualisation import *
from src.elements import create_pipe, create_elbow, setup_ifc_file

#create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
random.seed(10)

### CLOI Dataset Creation

The following section converts CLOI scans into a pcd dataset.

In [ ]:
# combine clouds
data_path = "/mnt/f/datasets/export/export/"
max_points = 4096

In [ ]:
classes = os.listdir(data_path)
print(classes)

In [ ]:
all_classes = []
element_count = 0
error_count = 0
for i, cl in enumerate(classes):
    all_elements = []
    class_path = data_path + cl
    elements = os.listdir(class_path)
    for j, el in tqdm(enumerate(elements)):
        try:
            element = np.loadtxt(class_path + '/' + el)

            # downsample
            if (len(element) > 0 and element.ndim == 2 and element.shape[1] == 4):
                if len(element)  > max_points:
                    #idx = np.random.randint(element.shape[0], size=max_points)
                    #element = element[idx :]
                    element = np.random.permutation(element)[:max_points]

                element = np.delete(element, 3, axis=1) # remove point index
                element = np.insert(element, 3, values=[element_count], axis=1) # add element index
                #print(element.shape)
                element_count += 1
                all_elements.append(element)
        except Exception as E:
            error_count += 1
            
            
    all_elements = np.vstack(all_elements)
    all_elements = np.insert(all_elements, 4, values=[i], axis=1) # add class index
    all_classes.append(all_elements)
    
all_classes = np.concatenate(all_classes)
print(all_classes.shape)
print ("errors: ", error_count)
        
        #print(points[0])

In [ ]:
pcd = o3d.t.geometry.PointCloud()
points = all_classes[:,0:3]
el_index  = [[i] for i in all_classes[:,3]]
cl_index  = [[i] for i in all_classes[:,4]]


pcd.point["positions"] = o3d.core.Tensor(points)
pcd.point["elements"] = o3d.core.Tensor(el_index)
pcd.point["classes"] = o3d.core.Tensor(cl_index)

o3d.t.io.write_point_cloud("water2.pcd", pcd)


## Pipe parameter detection

### Generation of synethetic IFC element dataset

#### dataset creation process
1. Generate params for element model
2. Generate ifc models
4. Convert to obj models using ifcConvert (ifc2obj.py script)
3. Convert to partially occluded EXR images using render_depth.py script
4. Covnert to point clouds using process_exr.py script
5. Combine multiple views of object to create training and testing datasets


##### Step 1 & 2. IFC model generation


In [4]:
density = 1024
sample_size = 32
config_path = "config/pipeline.json"
pcd_path = "/home/haritha/downloads/blender-2.79-linux-glibc219-x86_64/output/pcd/"
blueprint = 'data/sample.ifc'
num_scans = 16

In [5]:
"""
elbow - modelled as an IfcRevolvedAreaSolid 

params:
position - 3D coordinate
direction - 3D vector, axis of extrusion (normal to axis of revolution) (z>=0)
axis_position - 2D coordinate, relative to position
angle - angle of revolution (0 -> pi)
radius
"""


'\nelbow - modelled as an IfcRevolvedAreaSolid \n\nparams:\nposition - 3D coordinate\ndirection - 3D vector, axis of extrusion (normal to axis of revolution) (z>=0)\naxis_position - 2D coordinate, relative to position\nangle - angle of revolution (0 -> pi)\nradius\n'

In [6]:
"""
pipe - modelled as an IfcExtrudedAreaSolid

params:
position - 3D coordinate
extrusion_direction - 3D vector (z>=0)
length - 3D coordinate
radius
"""


'\npipe - modelled as an IfcExtrudedAreaSolid\n\nparams:\nposition - 3D coordinate\nextrusion_direction - 3D vector (z>=0)\nlength - 3D coordinate\nradius\n'

In [15]:
def synthetic_dataset(config, sample_size, element_class, output_base, blueprint, start=0):
    # setup
    f = open(config, 'r')
    config_data  = json.load(f)
    output_dir = os.path.join(output_base, element_class)
    #os.makedirs(output_dir)

    metadata = {}
    for i in tqdm(range(start, sample_size+start)):
        # generate ifc file
        ifc = setup_ifc_file(blueprint)
        owner_history = ifc.by_type("IfcOwnerHistory")[0]
        project = ifc.by_type("IfcProject")[0]
        context = ifc.by_type("IfcGeometricRepresentationContext")[0]
        floor = ifc.by_type("IfcBuildingStorey")[0]
        
        ifc_info = {"owner_history": owner_history,
            "project": project,
           "context": context, 
           "floor": floor}
        
        # generate ifc element
        if element_class == 'pipe':
            e = create_pipe(config_data[element_class], ifc, ifc_info)
        if element_class == 'elbow':
            e = create_elbow(config_data[element_class], ifc, ifc_info, blueprint)
    

        metadata[str(i)] = e
        ifc.write(os.path.join(output_dir, '%d.ifc' % i))
    
    with open(os.path.join(output_dir, 'metadata.json'), 'w') as f:
        json.dump(metadata, f)
        
        

In [16]:
synthetic_dataset(config_path, sample_size, "elbow", 'output', blueprint, 0)

  0%|          | 0/32 [00:00<?, ?it/s]

*Use external scripts to convert above IFC dataset into ocluded PCD dataset. (step 3, 4 & 5)*

##### Step 6. Test / train dataset creation

1. merge multiple views
2. sample to standard density
3. generate training and testing dataset



In [ ]:
def random_resample_cloud(points, density):
    indices = np.arange(points.shape[0])
    if (len(points) > density):
        sampled_indices = np.random.choice(indices, density, replace=False)
    else:
        sampled_indices = np.random.choice(indices, density, replace=True)
        
    #print(len(sampled_indices))
    return(points[sampled_indices])
        

In [ ]:
def save_cloud(points, output_base, name):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    save_path = os.path.join(output_base, name + ".pcd")
    o3d.io.write_point_cloud(save_path, pcd)

In [ ]:
def create_merged_dataset(pcd_path, output_base, element_class, num_scans, density, num_views=3, test_split=0.1):
    # load data
    metadata_file = os.path.join(output_base, element_class, 'metadata.json')
    f = open(metadata_file, 'r')
    metadata = json.load(f)
    
    scans = os.listdir(pcd_path)
    unique_files = set()
    for sc in scans:
        element = int(sc.split('_')[0])
        unique_files.add(element)
    
    # merge multiple views
    count = 0
    metadata_new = {}
    train_clouds = {}
    test_clouds = {}
    test_point = int(len(unique_files)*(1-test_split))
    print(test_point)
    for k, un in enumerate(unique_files):
        for i in range(num_scans-num_views):
            points = []
            for j in range(num_views):
                file_path = os.path.join(pcd_path, (str(un) + '_' + str(j) + '.pcd'))
                pcd = o3d.io.read_point_cloud(file_path)
                points.append(pcd.points)               

            #merged.points = o3d.utility.Vector3dVector(np.vstack(points))
            merged_points = np.vstack(points)
            metadata_new[str(count)] = metadata[str(un)]
            
            if k < test_point:
                train_clouds[str(count)] = merged_points
            else:
                test_clouds[str(count)] = merged_points
                
            count += 1

    # resample and save_data
    test_path = os.path.join(output_base, element_class, 'test')
    train_path = os.path.join(output_base, element_class, 'train')
    try:
        os.mkdir(test_path)
        os.mkdir(train_path)
    except:
        pass
    
    for k in train_clouds.keys():
        sampled_points = random_resample_cloud(train_clouds[k], density)
        save_cloud(sampled_points, train_path, k)
        
    for k in test_clouds.keys():
        sampled_points = random_resample_cloud(test_clouds[k], density)
        save_cloud(sampled_points, test_path, k)

    with open(os.path.join(output_base, element_class, 'metadata_new.json'), 'w') as f:
        json.dump(metadata_new, f)
        

In [ ]:
create_merged_dataset(pcd_path, 'output', 'pipe', num_scans, density, 3, 0.1)